In [15]:
import os
import time

import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


class InnerNode():

    def __init__(self, depth, args):
        self.args = args
        self.fc = nn.Linear(self.args.input_dim, 1)
        beta = torch.randn(1)
        #beta = beta.expand((self.args.batch_size, 1))
        if self.args.cuda:
            beta = beta.cuda()
        self.beta = nn.Parameter(beta)
        self.leaf = False
        self.prob = None
        self.leaf_accumulator = []
        self.lmbda = self.args.lmbda * 2 ** (-depth)
        self.build_child(depth)
        self.penalties = []

    def reset(self):
        self.leaf_accumulator = []
        self.penalties = []
        self.left.reset()
        self.right.reset()

    def build_child(self, depth):
        if depth < self.args.max_depth:
            self.left = InnerNode(depth+1, self.args)
            self.right = InnerNode(depth+1, self.args)
        else :
            self.left = LeafNode(self.args)
            self.right = LeafNode(self.args)

    def forward(self, x):
        return(F.sigmoid(self.beta*self.fc(x)))
    
    def select_next(self, x):
        prob = self.forward(x)
        if prob < 0.5:
            return(self.left, prob)
        else:
            return(self.right, prob)

    def cal_prob(self, x, path_prob):
        self.prob = self.forward(x) #probability of selecting right node
        self.path_prob = path_prob
        left_leaf_accumulator = self.left.cal_prob(x, path_prob * (1-self.prob))
        right_leaf_accumulator = self.right.cal_prob(x, path_prob * self.prob)
        self.leaf_accumulator.extend(left_leaf_accumulator)
        self.leaf_accumulator.extend(right_leaf_accumulator)
        return(self.leaf_accumulator)

    def get_penalty(self):
        penalty = (torch.sum(self.prob * self.path_prob) / torch.sum(self.path_prob), self.lmbda)
        self.penalties.append(penalty)
        if not self.left.leaf:
            left_penalty = self.left.get_penalty()
            right_penalty = self.right.get_penalty()
            
            self.penalties.extend(left_penalty)
            self.penalties.extend(right_penalty)
        return(self.penalties)


class LeafNode():
    def __init__(self, args):
        self.args = args
        self.param = torch.randn(self.args.output_dim)
        if self.args.cuda:
            self.param = self.param.cuda()
        self.param = nn.Parameter(self.param)
        self.leaf = True
        self.softmax = nn.Softmax()

    def forward(self):
        return(self.softmax(self.param.view(1,-1)))

    def reset(self):
        pass

    def cal_prob(self, x, path_prob):
        Q = self.forward()
        #Q = Q.expand((self.args.batch_size, self.args.output_dim))
        Q = Q.expand((path_prob.size()[0], self.args.output_dim))
        return([[path_prob, Q]])

    
    
    
    
    
    
    
    
    

class SoftDecisionTree(nn.Module):

    def __init__(self, args):
        super(SoftDecisionTree, self).__init__()
        self.args = args
        self.root = InnerNode(1, self.args)
        self.collect_parameters() ##collect parameters and modules under root node
        self.optimizer = optim.SGD(self.parameters(), lr=self.args.lr, momentum=self.args.momentum)
        self.test_acc = []
        self.define_extras(self.args.batch_size)
        self.best_accuracy = 0.0

    def define_extras(self, batch_size):
        ##define target_onehot and path_prob_init batch size, because these need to be defined according to batch size, which can be differ
        self.target_onehot = torch.FloatTensor(batch_size, self.args.output_dim)
        self.target_onehot = Variable(self.target_onehot)
        self.path_prob_init = Variable(torch.ones(batch_size, 1))
        if self.args.cuda:
            self.target_onehot = self.target_onehot.cuda()
            self.path_prob_init = self.path_prob_init.cuda()
    '''
    def forward(self, x):
        node = self.root
        path_prob = Variable(torch.ones(self.args.batch_size, 1))
        while not node.leaf:
            node, prob = node.select_next(x)
            path_prob *= prob
        return node()
    '''        
    def cal_loss(self, x, y):
        batch_size = y.size()[0]
        leaf_accumulator = self.root.cal_prob(x, self.path_prob_init)
        loss = 0.
        max_prob = [-1. for _ in range(batch_size)]
        max_Q = [torch.zeros(self.args.output_dim) for _ in range(batch_size)]
        for (path_prob, Q) in leaf_accumulator:
            TQ = torch.bmm(y.view(batch_size, 1, self.args.output_dim), torch.log(Q).view(batch_size, self.args.output_dim, 1)).view(-1,1)
            loss += path_prob * TQ
            path_prob_numpy = path_prob.cpu().data.numpy().reshape(-1)
            for i in range(batch_size):
                if max_prob[i] < path_prob_numpy[i]:
                    max_prob[i] = path_prob_numpy[i]
                    max_Q[i] = Q[i]
        loss = loss.mean()
        penalties = self.root.get_penalty()
        C = 0.
        for (penalty, lmbda) in penalties:
            C -= lmbda * 0.5 *(torch.log(penalty) + torch.log(1-penalty))
        output = torch.stack(max_Q)
        self.root.reset() ##reset all stacked calculation
        return(-loss + C, output) ## -log(loss) will always output non, because loss is always below zero. I suspect this is the mistake of the paper?

    def collect_parameters(self):
        nodes = [self.root]
        self.module_list = nn.ModuleList()
        self.param_list = nn.ParameterList()
        while nodes:
            node = nodes.pop(0)
            if node.leaf:
                param = node.param
                self.param_list.append(param)
            else:
                fc = node.fc
                beta = node.beta
                nodes.append(node.right)
                nodes.append(node.left)
                self.param_list.append(beta)
                self.module_list.append(fc)

    def train_(self, train_loader, epoch):
        self.train()
        self.define_extras(self.args.batch_size)
        for batch_idx, (data, target) in enumerate(train_loader):
            correct = 0
            if self.args.cuda:
                data, target = data.cuda(), target.cuda()
            #data = data.view(self.args.batch_size,-1)
            target = Variable(target)
            target_ = target.view(-1,1)
            batch_size = target_.size()[0]
            data = data.view(batch_size,-1)
            ##convert int target to one-hot vector
            data = Variable(data)
            if not batch_size == self.args.batch_size: #because we have to initialize parameters for batch_size, tensor not matches with batch size cannot be trained
                self.define_extras(batch_size)
            self.target_onehot.data.zero_()            
            self.target_onehot.scatter_(1, target_, 1.)
            self.optimizer.zero_grad()

            loss, output = self.cal_loss(data, self.target_onehot)
            loss.backward(retain_variables=True)
            self.optimizer.step()
            pred = output.data.max(1)[1] # get the index of the max log-probability
            correct += pred.eq(target.data).cpu().sum()
            accuracy = 100. * correct / len(data)

            if batch_idx % self.args.log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {}/{} ({:.4f}%)'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0],
                    correct, len(data),
                    accuracy))

    def test_(self, test_loader, epoch):
        self.eval()
        self.define_extras(self.args.batch_size)
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            if self.args.cuda:
                data, target = data.cuda(), target.cuda()
            target = Variable(target)
            target_ = target.view(-1,1)
            batch_size = target_.size()[0]
            data = data.view(batch_size,-1)
            ##convert int target to one-hot vector
            data = Variable(data)
            if not batch_size == self.args.batch_size: #because we have to initialize parameters for batch_size, tensor not matches with batch size cannot be trained
                self.define_extras(batch_size)
            self.target_onehot.data.zero_()            
            self.target_onehot.scatter_(1, target_, 1.)
            _, output = self.cal_loss(data, self.target_onehot)
            pred = output.data.max(1)[1] # get the index of the max log-probability
            correct += pred.eq(target.data).cpu().sum()
        accuracy = 100. * correct / len(test_loader.dataset)
        print('\nTest set: Accuracy: {}/{} ({:.4f}%)\n'.format(
            correct, len(test_loader.dataset),
            accuracy))
        self.test_acc.append(accuracy)

        if accuracy > self.best_accuracy:
            self.save_best('./result')
            self.best_accuracy = accuracy

    def save_best(self, path):
        try:
            os.makedirs('./result')
        except:
            print('directory ./result already exists')

        with open(os.path.join(path, 'best_model.pkl'), 'wb') as output_file:
            pickle.dump(self, output_file)

In [16]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

class myNestedImgDataset(Dataset):
    def __init__(self, dir_path, transform=None, test=False):
        self.dir_path = dir_path
        self.transform = transform
        self.classes = [x for x in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path,x))]
        self.img_paths = []
        self.labels = []
        self.test = test
        if self.test:
            class_img_paths = [os.path.join(dir_path,x) for x in os.listdir(dir_path)]
            self.img_paths.extend(class_img_paths)
        else:
            for class_idx, folder_name in enumerate(self.classes):
                prefix = os.path.join(dir_path,folder_name)
                class_img_paths = [os.path.join(prefix,x) for x in os.listdir(prefix)]
                self.img_paths.extend(class_img_paths)
                self.labels.extend(np.ones(len(class_img_paths))*class_idx)
            
            self.labels = [int(x) for x in self.labels]
            
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self,idx):
        if self.test:
            return torch.FloatTensor(plt.imread(self.img_paths[idx])), None
        else:
            return torch.FloatTensor(plt.imread(self.img_paths[idx])), self.labels[idx]
    
    def show(self,idx):
        return plt.imshow(mpimg.imread(self.img_paths[idx]), cmap='Greys')
    

class Img2FlatVec(Dataset):
    def __init__(self, dataset):
        self.dataset= dataset
        self.n = len(self.dataset)
        samp_img, _ = self.dataset[1]
        self.h, self.w = samp_img.shape
    
    def __getitem__(self,idx):
        x, y = self.dataset[idx]
        x_flat = x 
        return x.view(self.h*self.w)/255, y

    def __len__(self): return self.n

In [17]:
mnist_trn = myNestedImgDataset('/Users/timlee/data/MNIST/trn/')
trn_vec = Img2FlatVec(mnist_trn)
trn_dl = DataLoader(trn_vec, batch_size=8, shuffle=True, num_workers=4)
x_test, y_test = iter(trn_dl).next()
x_var = Variable(x_test)
y_var = y_test

In [21]:

from __future__ import print_function
import os
import argparse
import pickle
import torch
from torchvision import datasets, transforms


# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--input-dim', type=int, default=28*28, metavar='N',
                    help='input dimension size(default: 28 * 28)')
parser.add_argument('--output-dim', type=int, default=10, metavar='N',
                    help='output dimension size(default: 10)')
parser.add_argument('--max-depth', type=int, default=8, metavar='N',
                    help='maximum depth of tree(default: 3)')
parser.add_argument('--epochs', type=int, default=1, metavar='N',
                    help='number of epochs to train (default: 40)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--lmbda', type=float, default=0.1, metavar='LR',
                    help='temperature rate (default: 0.1)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')

args = parser.parse_args(args=[])
args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

try:
    os.makedirs('./data')
except:
    print('directory ./data already exists')

kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)

def save_result(acc):
    try:
        os.makedirs('./result')
    except:
        print('directory ./result already exists')
    filename = os.path.join('./result/', 'bp_deep.pickle' if args.deep else 'bp.pickle')
    f = open(filename,'w')
    pickle.dump(acc, f)
    f.close()

model = SoftDecisionTree(args)

if args.cuda:
    model.cuda()

for epoch in range(1, args.epochs + 1):
    model.train_(train_loader, epoch)
    model.test_(test_loader, epoch)
#save_result()

directory ./data already exists


/Users/timlee/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:85: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/timlee/anaconda2/envs/py3/lib/python3.6/site-packages/torch/autograd/__init__.py:93: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "


Train Epoch: 1 [0/60000 (0%)]	Loss: 3.029751, Accuracy: 5/64 (7.8125%)
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.998362, Accuracy: 8/64 (12.5000%)
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.988293, Accuracy: 8/64 (12.5000%)
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.965191, Accuracy: 4/64 (6.2500%)
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.946007, Accuracy: 7/64 (10.9375%)
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.905143, Accuracy: 11/64 (17.1875%)
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.884119, Accuracy: 8/64 (12.5000%)
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.851951, Accuracy: 18/64 (28.1250%)
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.842857, Accuracy: 16/64 (25.0000%)
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.818438, Accuracy: 11/64 (17.1875%)
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.741905, Accuracy: 17/64 (26.5625%)
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.710649, Accuracy: 14/64 (21.8750%)
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.685089, Accuracy: 14/64 (21.8750%)
Train Epoch: 1 [8320

In [ ]:
# for x in model.parameters():
#     print(x)

In [23]:
model.root.cal_prob()

TypeError: cal_prob() missing 2 required positional arguments: 'x' and 'path_prob'

In [34]:
mydata,y = iter(train_loader).next()

In [35]:
mydata.view(64,-1)


-0.4242 -0.4242 -0.4242  ...  -0.4242 -0.4242 -0.4242
-0.4242 -0.4242 -0.4242  ...  -0.4242 -0.4242 -0.4242
-0.4242 -0.4242 -0.4242  ...  -0.4242 -0.4242 -0.4242
          ...             ⋱             ...          
-0.4242 -0.4242 -0.4242  ...  -0.4242 -0.4242 -0.4242
-0.4242 -0.4242 -0.4242  ...  -0.4242 -0.4242 -0.4242
-0.4242 -0.4242 -0.4242  ...  -0.4242 -0.4242 -0.4242
[torch.FloatTensor of size 64x784]

In [41]:
model.root.cal_prob(Variable(mydata.view(64,-1)), Variable(torch.ones(64,1)))

/Users/timlee/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:85: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [38]:
model.path_prob_init

Variable containing:
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
    1
[torch.FloatTensor of size 16x1]